#Intelligent Systems - Assignment 4 - Machine Learning

- Member Names: **Mireya Quintana and Haizea Rodriguez**
- Group ID: **C**
- Assignment: **04**
- Date: **20/05/2021**
- Time spent in the assignment: **7h**
- Proportion of effort done by members of the group: **50/50**
- Doubts and difficulties that arose during the realization:


---

**General instructions**
 1. If you use a function of a certain library, do not forget to include in the packages vector
 2. Do not forget to comment on the code, especially those non-trivial commands (part of the rating depends on the cleaning of the code)
 3. It is strongly recommended to test any implemented function in order to    check for its proper operation

---

 **Submission**
- Print the notebook in a PDF file with all the sections expanded and all the cells executed, in a way all the code can be seen, and all the results are shown.
- Save the notebook as ipynb file.
- Submit both files, in addition to any input file needed for its execution in a zip folder into ALUD

---

**Evaluation**
- Total Evaluation - 25%
  - Correct and error-free execution - 15%
  - Documentation of analysis of results - 10%

---
**Deadline**
May 20th

We have included some libraries in order to show the correlaion matrix. The loading of them it is quite slow. 

In [ ]:
# PLEASE, INCLUDE HERE IF YOU USE ANY ADDITIONAL LIBRARY
packages = c("glue","ggplot2","gridExtra","tidyverse","caret","rpart","e1071","readr", "Matrix", "reshape2", "dplyr", "tidyr", "ggvis", "corrplot")
newpack  = packages[!(packages %in% installed.packages()[,"Package"])]
if(length(newpack)) install.packages(newpack)
a=lapply(packages, library, character.only=TRUE)
rm(list = ls())

## Linear Regression




### Data Description 📚
In this part of the assignment you are going to answer the question **How accurately can average, minimum and maximum temperature be predicted from historical data?**

We will analyse the margin of error when predicting the minimum, average and maximum temperature based on historical data.

We will use a [CSV](https://drive.google.com/open?id=1A6Z0Azqu-c-mFDoFcmbCvnGeylHeby1N&authuser=enrique.onieva%40deusto.es&usp=drive_fs) available in the Drive folder with data from the [AEMET OpenData service](http://www.aemet.es/es/datos_abiertos/AEMET_OpenData) (*loiu_2000-01_2021-03.csv*).

Loiu Airport weather station between 1 January 2010 and 31 March 2021.
The dataset has 4018 records of 20 attributes ([see description here](https://docs.google.com/spreadsheets/d/1pVy1eDi2YEUzetP0cEbxDz09uuDkQ7kUIgremgyNdnM/edit#gid=1743573338)). 


###  You are asked to: 🚨
1. Read the provided file and store in a data.frame
2. Select up to 5 columns (without the target) to make predictions.
3. Repeat 10 times:
  - Randomly divide data in train (80%) and test (20%)
    - 🔎 For this, you can use the [createDataPartition](https://www.rdocumentation.org/packages/caret/versions/6.0-86/topics/createDataPartition) function of the *caret* package as:
    - createDataPartition(y = **data$Target**, p = **percentage**, list = FALSE)
  - Train a linear regression model
    - 🔎 For this, you have to use the [lm](https://www.rdocumentation.org/packages/stats/versions/3.6.2/topics/lm) function as:
    - model = lm(**Target~xxxxxxx**, **training.data**)
  - Make predictions over the test data
    - 🔎 You will have to use the predict function as:
    - predictions = predict(**model**,**test.data**)
  - Calculate and print Mean Absolute Error (MAE)
    - 🔎 You can calculate MAE as:
    - mean(abs(prediction - test_data$Target))

**(You will have to do this procedure three times: one for prediction of average, other minimum and the third one for maximum temperature)**

**You have to present:**
- Justification  used to select the attributes for each one of the predictions.
- Table with MAE results of the 10 iterations
- Average MAE and Standar Deviation get from the 10 iterations 

### Your solution here 💻

####Reading and cleaning data

The CSV uploaded to our gitHub repository.
The data contains information about the weather in Loiu from 2010 to 2021


In [ ]:
# Read the csv file and store it into a data.frame
fileLoiu = "https://raw.githubusercontent.com/HaizeaR/Sistemas-Inteligentes-Files/main/loiu_2000-01_2021-03.csv"
LoiuData = read.csv(fileLoiu)
knitr::kable(head(LoiuData), caption = "A glipmse of the data")

One of the most important parts of working with sets  of data is preparing it for the exploration. The first step traken after reading the file is visualizing a summary of all the variables indicating the type as well. 

In [ ]:
#This will show a glipse of the values we have
summary(LoiuData)
#This will indicate the type of each variable
str(LoiuData)

Once we have understood what we are working with the 1st task will be changing the type of some variables.


>In the case of **hours** we have "strange" values. The word "Varias" appears several times. Our approach has been changin it to 0 with the following code: 








In [ ]:
#Remove all the values that may give us problems 
#Varias could generate Nan's so we will change it to 0 

LoiuData$horatmin=ifelse(LoiuData$horatmin=="Varias",0, as.character(LoiuData$horatmin))
LoiuData$horatmax=ifelse(LoiuData$horatmax=="Varias",0, as.character(LoiuData$horatmax))
LoiuData$horaPresMin=ifelse(LoiuData$horaPresMin=="Varias",0, as.character(LoiuData$horaPresMin))
LoiuData$horaracha=ifelse(LoiuData$horaracha=="Varias",0, as.character(LoiuData$horaracha))

>Then we have decided to stay with just the hour part. If the store value is 21:36 we will keep 21. 

>After doing all the procedure we will change the Hours to integer, so we can evaluate them as numbers. 

In [ ]:
# The SUB will remove all the not needed parts giving us the option of keeping just the H part
# If data is H:M we will just work with H which refers to Hours

LoiuData$horaPresMin = as.factor(LoiuData$horaPresMin)
LoiuData$horaPresMin = sub("\\:.*", "",LoiuData$horaPresMin)
LoiuData$horaPresMin = as.integer(LoiuData$horaPresMin)
#LoiuData$horaPresMin = na.omit(LoiuData$horaPresMin)

LoiuData$horaracha = as.factor(LoiuData$horaracha)
LoiuData$horaracha = sub("\\:.*", "",LoiuData$horaracha)
LoiuData$horaracha = as.integer(LoiuData$horaracha)
#LoiuData$horaracha = na.omit(LoiuData$horaracha)

LoiuData$horatmax = as.factor(LoiuData$horatmax)
LoiuData$horatmax = sub("\\:.*", "", LoiuData$horatmax)
LoiuData$horatmax = as.integer(LoiuData$horatmax)
#LoiuData$horatmax = na.omit(LoiuData$horatmax)

LoiuData$horatmin = as.character(LoiuData$horatmin)
LoiuData$horatmin = sub("\\:.*", "", LoiuData$horatmin)
LoiuData$horatmin = as.integer(LoiuData$horatmin)
#LoiuData$horatmin = na.omit(LoiuData$horatmin)


> We will **create a new variable** "mes". We believe that it could be interesting to know the month to calculate the temperature and we will test if our prediction is correct later on. 


In [ ]:
# CREATE A NEW VARIABLE 
LoiuData$mes <- substr(LoiuData$fecha,6,7)
LoiuData$mes = as.integer(LoiuData$mes)

We change the type os some variables to factor in order to know if they are a collection of values or it is always the same one. 

In [ ]:
LoiuData$fecha = as.factor(LoiuData$fecha) 
LoiuData$nombre = as.factor(LoiuData$nombre) 
LoiuData$provincia = as.factor(LoiuData$provincia)

> We are going to erase all the NaN's because keeping them may create problems

In [ ]:
LoiuData = LoiuData %>% drop_na()

In [ ]:
# Verify that all the cahnges are done correctly
summary(LoiuData)
str(LoiuData)
knitr::kable(head(LoiuData), caption = "A glipmse of the data")

> **Erase** all the columns that we considerer that are not relevant for the evaluation

> **Nombre, Provincia, Indicativo and Altitud** have always the same value.
> **Hora Pres Max** has a lot of "Varias" values and all the rest are either 24 or 0 so we are going to remove it because we believe it does not provide a great deal of useful information.

In [ ]:
LoiuData = LoiuData %>% select(-nombre,-provincia, -indicativo, -altitud, -horaPresMax)
#we have decided to get rid off horapremax because most values were "Varias" which do not give us a lot of infomation

In [ ]:
knitr::kable(head(LoiuData), caption = "A glipmse of the data")

In [ ]:
str(LoiuData)
summary(LoiuData)

####Correlation matrix

DEFINITION: 

A correlation matrix is a table which displays the correlation coefficients for different variables. The matrix depicts the correlation between all the possible pairs of values in a table.

A correlation matrix consists of rows and columns that show the variables. Each cell in a table contains the correlation coefficient.



We are going to use the correlation matrix to obtain the 5 best variables for each case, tmax, tmin ant tavg.

Variables can be directly (blue) or indirectly (red) correlated and those ones closer to 1 or -1 will be the most interesting ones. 



To calculate the correlation matrix only numerical values can be used this is the reason why we only calculate for the columns that go from tmed to mes 

In [ ]:
# Only numerical values can be correlated
correlation<- LoiuData %>% select(tmed:mes)

M <- cor(correlation) # correlation matrix

# By changing the method parameter the type of visualization changes 
# “circle”, “square”, “ellipse”, “number”, “shade”, “color”, “pie”
corrplot(M, method = "number")

RESULTS of correlation matrix 

>TMAX 

> - tmed(0.95)
> - tmin(0.77)
> - sol(0.56)
> - prec(-0.33)
> - mes(0.27)

> TMIN
> - tmed(0.93)
> - tmax(0.77)
> - mes(0.3)
> - presMax(-0.25)
> - prec (-0.14)

>TAVG
> - tmax(0.95)
> - tmin(0.93)
> - sol(0.35)
> - mes(0.30)
> - prec(-0.25)



*(values obtain looking at the row of the Target variable)*


We believe that the obtained variables are the ones that are truthly related, it makes total sense that the sun will be an important variable when trying to predict the temperature, as well as the others.

####MAX

In [ ]:
LoiuDataMAX = data.frame(LoiuData$tmax, LoiuData$tmed, LoiuData$tmin, LoiuData$sol, LoiuData$prec, LoiuData$mes)
knitr::kable(head(LoiuDataMAX))

valuesMAEmax =c()

#Repeat the procces 10 times
for(i in 1:10){
  print(glue("--------------------------------------------------"))
  print(glue("Number of iteration: {i}"))

  knitr::kable(head(LoiuDataMAX), caption = "A glipmse of the data")

  # Get indexes to do partitions
  perc = 0.80
  index.train = caret::createDataPartition(y = LoiuDataMAX$LoiuData.tmax, p = perc, list = FALSE)

  # Getting partitions
  data.train = LoiuDataMAX[ index.train, ]
  data.test  = LoiuDataMAX[-index.train, ]
  print(glue(""))
  print(glue("Size of the training set: {nrow(data.train)}"))
  print(glue("Size of the test set: {nrow(data.test)}"))

  # Create Linear Model using training data. Formula = all the columns except Score
  model = lm(formula = LoiuData.tmax ~., data = data.train)
  print(model)

  # Make the prediction using the model and test data
  prediction = predict(model, data.test)
 
  #print(summary(prediction)) # we have found that the prediction contained some NAs we are going to erase them so we obtain a mean value
  prediction = na.omit(prediction)
  # Calculate Mean Average Error
  mean_abs_error = mean(abs(prediction - data.test$LoiuData.tmax))
 
  print(glue("Mean Absolute Error: {mean_abs_error}"))

  valuesMAEmax[i] = mean_abs_error

}
print(glue(""))
print(glue("......................."))
print(glue("RESULTS"))
knitr::kable(valuesMAEmax)
print(glue("Mean MAE: {mean(valuesMAEmax)}"))
print(glue("Standard Deviation of MAE: {sd(valuesMAEmax)}"))

####MIN

In [ ]:
LoiuDataMIN = data.frame(LoiuData$tmin, LoiuData$tmed, LoiuData$tmax, LoiuData$mes, LoiuData$presMax, LoiuData$prec)
knitr::kable(head(LoiuDataMIN))

valuesMAEmin =c()

#Repeat the procces 10 times
for(i in 1:10){
  print(glue("--------------------------------------------------"))
  print(glue("Number of iteration: {i}"))

  knitr::kable(head(LoiuDataMIN), caption = "A glipmse of the data")

  # Get indexes to do partitions
  perc = 0.80
  index.train = caret::createDataPartition(y = LoiuDataMIN$LoiuData.tmin, p = perc, list = FALSE)

  # Getting partitions
  data.train = LoiuDataMIN[ index.train, ]
  data.test  = LoiuDataMIN[-index.train, ]
  print(glue(""))
  print(glue("Size of the training set: {nrow(data.train)}"))
  print(glue("Size of the test set: {nrow(data.test)}"))

  # Create Linear Model using training data. Formula = all the columns except Score
  model = lm(formula = LoiuData.tmin ~., data = data.train)
  print(model)

  # Make the prediction using the model and test data
  prediction = predict(model, data.test)
 
  #print(summary(prediction)) # we have found that the prediction contained some NAs we are going to erase them so we obtain a mean value
  prediction = na.omit(prediction)
  # Calculate Mean Average Error
  mean_abs_error = mean(abs(prediction - data.test$LoiuData.tmin))
 
  print(glue("Mean Absolute Error: {mean_abs_error}"))

  valuesMAEmin[i] = mean_abs_error

}
knitr::kable(valuesMAEmin)

print(glue("......................."))
print(glue("Mean MAE: {mean(valuesMAEmin)}"))
print(glue("Standard Deviation of MAE: {sd(valuesMAEmin)}"))

####AVERAGE

In [ ]:
LoiuDataAVG = data.frame(LoiuData$tmed, LoiuData$tmax, LoiuData$tmin, LoiuData$sol, LoiuData$mes, LoiuData$prec)
knitr::kable(head(LoiuDataAVG))

valuesMAEavg =c()

#Repeat the procces 10 times
for(i in 1:10){
  print(glue("--------------------------------------------------"))
  print(glue("Number of iteration: {i}"))

  knitr::kable(head(LoiuDataAVG), caption = "A glipmse of the data")

  # Get indexes to do partitions
  perc = 0.80
  index.train = caret::createDataPartition(y = LoiuDataAVG$LoiuData.tmed, p = perc, list = FALSE)

  # Getting partitions
  data.train = LoiuDataAVG[ index.train, ]
  data.test  = LoiuDataAVG[-index.train, ]
  print(glue(""))
  print(glue("Size of the training set: {nrow(data.train)}"))
  print(glue("Size of the test set: {nrow(data.test)}"))

  # Create Linear Model using training data. Formula = all the columns except Score
  model = lm(formula = LoiuData.tmed ~., data = data.train)
  print(model)

  # Make the prediction using the model and test data
  prediction = predict(model, data.test)
 
  #print(summary(prediction)) # we have found that the prediction contained some NAs we are going to erase them so we obtain a mean value
  prediction = na.omit(prediction)
  # Calculate Mean Average Error
  mean_abs_error = mean(abs(prediction - data.test$LoiuData.tmed))
 
  print(glue("Mean Absolute Error: {mean_abs_error}"))

  valuesMAEavg[i] = mean_abs_error

}
knitr::kable(valuesMAEavg)

print(glue("......................."))
print(glue("Mean MAE: {mean(valuesMAEavg)}"))
print(glue("Standard Deviation of MAE: {sd(valuesMAEavg)}"))

Write your analysis or answers to questions here (use as many cells  as needed)

#### RESULTS

This is done just with the purpose of visualizing everything together 

In [ ]:
print(glue("The results of the 3 cases "))
print(glue("..........................."))

results = data.frame(valuesMAEmax,valuesMAEmin,valuesMAEavg)
knitr::kable(results)
print(glue(""))
print(glue("MAX"))
print(glue("................"))
print(glue("Mean MAE: {mean(valuesMAEmax)}"))
print(glue("Standard Deviation of MAE: {sd(valuesMAEmax)}"))
print(glue(""))
print(glue("MIN"))
print(glue("................"))
print(glue("Mean MAE: {mean(valuesMAEmin)}"))
print(glue("Standard Deviation of MAE: {sd(valuesMAEmin)}"))
print(glue(""))
print(glue("AVG"))
print(glue("................"))
print(glue("Mean MAE: {mean(valuesMAEavg)}"))
print(glue("Standard Deviation of MAE: {sd(valuesMAEavg)}"))

### An End-to-End example 👀
(but please, in your solution do not do like me and use more than one cell to organize the code, when possible 🙂)

In [ ]:
data = read.csv(file="https://raw.githubusercontent.com/HaizeaR/Sistemas-Inteligentes-Files/main/2020%20-%202019%20world%20happiness.csv", sep=",", header = TRUE)
# Remove unuseful columns
data = data %>% select(-Country.or.region,-Overall.rank)

knitr::kable(head(data), caption = "A glipmse of the data")

# Get indexes to do partitions
perc = 0.80
index.train = createDataPartition(y = data$Score, p = perc, list = FALSE)

# Getting partitions
data.train = data[ index.train, ]
data.test  = data[-index.train, ]
print(glue(""))
print(glue("Size of the training set: {nrow(data.train)}"))
print(glue("Size of the test set: {nrow(data.test)}"))

# Create Linear Model using training data. Formula = all the columns except Score
model = lm(formula = Score ~., data = data.train)
print(model)

# Make the prediction using the model and test data
prediction = predict(model, data.test)

# Calculate Mean Average Error
mean_abs_error = mean(abs(prediction - data.test$Score))
print(glue("Mean Absolute Error: {mean_abs_error}"))

## Decision Trees

### Data Description 📚
[Divorce prediction](https://archive.ics.uci.edu/ml/datasets/Divorce+Predictors+data+set). Within this assignment, you will **create a decision tree able to predict if a couple will divorce or not** depending of answer given to a standard psicological form.

Data from 190 participants (49% divorced and 51% married) is available in the [Drive folder](https://drive.google.com/drive/folders/1A6Z0Azqu-c-mFDoFcmbCvnGeylHeby1N). In the folder, you also have an excel file with questions in the form. 


### You are asked to: 🚨

1. Read the provided file and store in a data.frame
2. Repeat 10 times
  - Randomly divide data in train (80%) and test (20%)
    - 🔎 For this, you can use the [createDataPartition](https://www.rdocumentation.org/packages/caret/versions/6.0-86/topics/createDataPartition) function of the *caret* package as:
    - createDataPartition(y = **data$Target**, p = **percentage**, list = FALSE)
  - Train a decision tree to predict the divorce given the responses
    - 🔎 For this, you have to use the [rpart](https://www.rdocumentation.org/packages/rpart/versions/4.1-15/topics/rpart) function within the *caret* package as:
    - model = model(**Target~xxxxxxx**, **training.data**)
  - Make predictions over the test data
    - 🔎 You will have to use the predict function as:
    - predictions = predict(**model**,**test.data**)
  - Calculate and print the accuracy of the model over the test data.
    - The accuracy is calculated as the percertage of hit over the giving data, or the percentage of data in which the real class is equal to the predicted one.
3. After the loop, build a decision tree with all the data and print it.
4. Explore the *tree* object in order to print the top-5 most relevant questions to predict if a couple will end in divorce or not. 

**You have to present**
- Top-5 most relevant questions.
- A printed version of the obtained tree.
- A table with the accuracy results for the 10 runs.
- Average and standard deviation of the accuracies obtained.


###Your solution here 💻

In [ ]:
DivorceFile = "https://raw.githubusercontent.com/HaizeaR/Sistemas-Inteligentes-Files/main/divorce.csv"
DivorceData = read.csv(DivorceFile, header = TRUE, fill = TRUE, sep = ';')
knitr::kable(head(DivorceData), caption = "A glipmse of the data")


With the aim of knowing how many times a value appears we think that it is interseting to change the type from numeric to FACTOR.

In [ ]:
cols <- c("Atr1","Atr2","Atr3","Atr4","Atr5","Atr6","Atr7","Atr8","Atr9","Atr10","Atr11","Atr12","Atr13","Atr14","Atr15","Atr16","Atr17","Atr18","Atr19","Atr20","Atr21","Atr22","Atr23","Atr24","Atr25","Atr26","Atr27","Atr28","Atr29","Atr30","Atr31","Atr32","Atr33","Atr34","Atr35","Atr36","Atr37","Atr38","Atr39","Atr40","Atr41","Atr42","Atr43","Atr44","Atr45","Atr46","Atr47","Atr48","Atr49","Atr50","Atr51","Atr52","Atr53","Atr54","Class")
# LAPPLY - apply the operation to all 
#In this case change to factor all the columns of DivorceData
 
DivorceData$Class=ifelse(DivorceData$Class==0,"D", "ND")
DivorceData[cols] <- lapply(DivorceData[cols], factor)
summary(DivorceData)

In [ ]:
overall = c()
questions = list()

for(i in 1:10){
  print(glue("Iteration: {i}"))
  print(glue("--------------------------"))
  # Get indexes to do partitions
  perc = 0.80
  index.train = createDataPartition(y = DivorceData$Class, p = perc, list = FALSE)

  # Getting partitions
  data.train = DivorceData[ index.train, ]
  data.test  = DivorceData[-index.train, ]
  print(glue(""))
  print(glue("Size of the training set: {nrow(data.train)}"))
  print(glue("Size of the test set: {nrow(data.test)}"))

  # Create Linear Model using training data. Formula = all the columns except Score
  model = rpart(formula = Class ~., data = data.train)
  

  questions$names[i] = names(model$variable.importance[1])
  questions$value[i] = model$variable.importance[1]

  print(model)
  #plot(model)
  #text(model, digits = 3)
  
  # Make the prediction using the model and test data
  prediction = predict(model, data.test, type= "class")
  
  # Display Confusion Matrix
  print(glue(""))
  mat = caret::confusionMatrix(data.test$Class, prediction)
  print(mat$table)

  # Print the accuracy 
  print(glue(""))
  print(glue("The model hits in {round(100*mat$overall[[1]],2)} % of the couples"))
 # DivorceData =   DivorceData %>% select(-)

  overall[i] = round(100*mat$overall[[1]],2)

}


In [ ]:
print(glue("The results after the 10 iterations are the following:"))
print(glue("- - - - - - - - - - - - - - - - - - - - - - - - - - - - -"))
print(glue(""))
print(glue("For each iteration the model hits the following %:"))
knitr::kable(overall)
print(glue(""))
print(glue("Average: {mean(overall)}"))
print(glue("Standard deviation: {sd(overall)}"))
print(glue(""))
print(glue("This are the most relevant questions and the importance value of each of them:"))
questionsResults = data.frame(questions$names, questions$value)
knitr::kable(questionsResults)

The most important questions are **18, 11** because they always appear on the list. 
There are other questions that sometimes appear and other time they do not but we can consider them important as well. (19,16,17)

The questions: 
>- 18. My spouse and I have similar ideas about how marriage should be 
>- 11. I think that one day in the future, when I look back, I see that my spouse and I have been in harmony with each other.  
>- 19. My spouse and I have similar ideas about how roles should be in marriage 
>- 16. We're compatible with my spouse about what love should be. 
>- 17. We share the same views about being happy in our life with my spouse 

### An End-to-End example 👀
(but please, in your solution do not do like me and use more than one cell to organize the code, when possible 🙂)

In [ ]:
data = read.table(file="https://raw.githubusercontent.com/HaizeaR/Sistemas-Inteligentes-Files/main/2020%20covid-19-recommendations.tab", header = T, stringsAsFactors = T)
# Remove unuseful columns
knitr::kable(head(data), caption = "A glipmse of the data")

# Get indexes to do partitions
perc = 0.80
index.train = createDataPartition(y = data$TARGET, p = perc, list = FALSE)

# Getting partitions
data.train = data[ index.train, ]
data.test  = data[-index.train, ]
print(glue(""))
print(glue("Size of the training set: {nrow(data.train)}"))
print(glue("Size of the test set: {nrow(data.test)}"))

# Create Linear Model using training data. Formula = all the columns except Score
model = rpart(formula = TARGET ~., data = data.train)
print(model)

# Make the prediction using the model and test data
prediction = predict(model, data.test, type= "class")

# Display Confusion Matrix
print(glue(""))
mat = caret::confusionMatrix(data.test$TARGET, prediction)
mat$table

# Print the accuracy 
print(glue(""))
print(glue("The model hits in {round(100*mat$overall[[1]],2)} % of the patients"))